This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [62]:
# get api key from your .env file
import os
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv('NASDAQ_API_KEY')

print("API_KEY:", API_KEY)

API_KEY: mQ57PNpohUGVC_utW71N


Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [63]:
# First, import the relevant modules
import requests

Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [64]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
url = 'https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data.json'

params = dict(key=API_KEY,start_date='2020-11-30',end_date='2020-12-01')

res = requests.get(url,params)

json_data = res.json()

for k in json_data.keys():
    print(k + ': ', json_data[k])

dataset_data:  {'limit': None, 'transform': None, 'column_index': None, 'column_names': ['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover'], 'start_date': '2020-11-30', 'end_date': '2020-12-01', 'frequency': 'daily', 'data': [['2020-12-01', 112.2, 112.2, 111.5, 112.0, None, 51.0, 5703.0, None, None, None], ['2020-11-30', 111.0, 113.6, 111.0, 112.1, None, 315.0, 35111.5, None, None, None]], 'collapse': None, 'order': None}


In [65]:
json_data['dataset_data']['data'][0][0]

'2020-12-01'

In [66]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

**Collecting data from FSE for AFX_X for the whole of 2017**

In [67]:
#Collect AFX_X 2017 data

params_2017 = dict(key=API_KEY,start_date='2017-01-01',end_date='2017-12-31')

AFX_X_2017 = requests.get(url,params_2017)

json_AFX_X_2017 = AFX_X_2017.json()

#json_AFX_X_2017 is in Python dict format

**Calculating the highest and lowest opening prices of 2017**

In [68]:
#To calculate the highest and lowest opening prices in the period,
#create a new list that contains just all open prices.

data_list = json_AFX_X_2017['dataset_data']['data']

opening_prices = []

for data in data_list:
    opening_prices.append(data[1])

#Removing non-floats from the list
opening_prices = [item for item in opening_prices if isinstance(item, float)]

    

In [69]:
#Find max and min in opening_prices

max_price = max(opening_prices)
min_price = min(opening_prices)

max_price_date = [item[0] for item in data_list if item[1] == max_price][0]

#print(max_price_date)

min_price_date = [item[0] for item in data_list if item[1] == min_price][0]

#print(min_price_date)

print('The highest opening price of 2017 was '+ str(max_price) + ', which was on ' + str(max_price_date) +'.')
print('The lowest opening price of 2017 was '+ str(min_price) + ', which was on ' + str(min_price_date) + '.')

The highest opening price of 2017 was 53.11, which was on 2017-12-14.
The lowest opening price of 2017 was 34.0, which was on 2017-01-24.


**The Largest Intraday Price Change**
The instruction asks to base the calculation on daily high and low, so I will follow it, but there is also 'Change' values provided by the API, so it would be simpler to just looked for the highest aboslute "Change" value in the dataset.

In [74]:
#Create a nested list that houses lists containing daily highs and lows.

high_low_list = []

for price in data_list:
    high_low_list.append([price[0],price[2], price[3]])
    
#print(high_low_list)

In [75]:
#Calculate the absolute difference between the high, low prices in each nested list
daily_change = []

for price in high_low_list:
    daily_change.append([price[0],abs(price[1]-price[2])])
    
#print(daily_change)

In [72]:
#Find the largest value in the daily_change list
max_daily_change = max(daily_change, key=lambda x: x[1])

print('The largest daily price change occurred on ' + str(max_daily_change[0]) + ', and was ' + str(round(max_daily_change[1],2)) + '.')

The largest daily price change occurred on 2017-05-11, and was 2.81.


**What was the largest change between any two days (based on Closing Price)?**

In [73]:
#Create a nested list where the sublist contain 2 consecutive dates and 2 closing prices

#To do this, first reduce data_list to a nested list with sublists that only has dates and closing price.

close_price = []

for data in data_list:
    close_price.append([data[0],data[4]])
    
#print(close_price)    

#Next loop through close_price to create a list with sublist that contains [date, date+1, close_price of date, close_price of date+1]

close_price_2d = []

for i in range(len(close_price)-1):
    new_sublist = [close_price[i][0],close_price[i+1][0],close_price[i][1],close_price[i+1][1]]
    close_price_2d.append(new_sublist)
    
#print(close_price_2d)

#Next loop through close_price_2d and create a new list with 2 dates and the difference between the two close prices in each sublist

price_change_2d = []

for price in close_price_2d:
    price_change_2d.append([price[0], price[1], (price[2]-price[3])])
    
#print(price_change_2d)

#Lastly find the largest absolute price change in price_change_2d

max_change_2d = max(price_change_2d, key=lambda x: abs(x[2]))

print('The largest close price difference was ' + str(round(max_change_2d[2],2)) + ' between ' + str(max_change_2d[1]) + \
      ' and ' + str(max_change_2d[0]) + '.')

The largest close price difference was -2.56 between 2017-08-08 and 2017-08-09.


**6.What was the average daily trading volume during this year?**

In [80]:
#calculate average of data_list[6]

sum_volume = 0

for volume in data_list:
    sum_volume += volume[6]

#print(sum_volume)

average_daily_trading_volume = sum_volume / len(data_list)

print(average_daily_trading_volume)


22726706.0
89124.33725490196


**7. Optional - what is the median of the traded volume**

In [95]:
#Create a list of Traded Volume and sort them

traded_volume = []

for data in data_list:
    traded_volume.append(data[6])
    
traded_volume = sorted(traded_volume)

#print(traded_volume)

#create a list of index to go with each traded volume item

index_list = list(range(len(traded_volume)))

#combine index and traded volume lists into a dict

index_volume_dict = {key: value for key, value in zip(index_list,traded_volume)}

#print(index_volume_dict)

#Calculate median index as key, and get the corresponding value.

import statistics

median_index = statistics.median(index_list)

median_traded_volume = index_volume_dict.get(median_index)

print(median_traded_volume)
  

76286.0
